In [ ]:
#BFACTORS
!pip install biopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 878 kB 4.2 MB/s 


In [ ]:
#Import Biopandas
from biopandas.pdb import PandasPdb
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
ppdb = PandasPdb()

In [ ]:
# Reading PDB files
#Creating bfactor dataframe of concatenated replicates for both conditions
def bfactor(path, file_ext):
  all_files = glob.glob(os.path.join(path, '*.pdb')) # list of all files in the current path
  each_file = (ppdb.read_pdb(f) for f in all_files)
  df_each_bfac_file = (ppdb.df['ATOM'][['residue_number', 'residue_name','b_factor']].drop_duplicates() for f in each_file)
  bfactor = pd.concat(df_each_bfac_file, ignore_index=True)
  return(bfactor, df_each_bfac_file)

bfactor_1 = bfactor('./bfactor', '*.xvg')[0]
bfactor_1_all = bfactor('./bfactor', '*.xvg')[1]

In [ ]:
#Generatting xls with bfactor for protein model UHL831
bf1_mean = bfactor_1.groupby('residue_number').mean().reset_index() #bfactor_1 is the average of different bfactor files
bf1_mean.columns = ['residue_number', 'b_factor2']

#Reading PDB files
pdb_file = ppdb.read_pdb('./input_831_bfactor.pdb')
pdb_file.df['ATOM'] = pdb_file.df['ATOM'].merge(bf1_mean, on = 'residue_number')
pdb_file.df['ATOM']['b_factor'] = pdb_file.df['ATOM']['b_factor2']

pdb_file.df['ATOM'].drop(columns = ['b_factor2'], inplace = True)

pdb_file.to_pdb(path='./input_831_bfactor.pdb', records=None, gz=False, append_newline=True)

pdb_file.df['ATOM']
#bf1_mean

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1,,N,,LEU,,,1,,...,61.569,82.012,18.764,1.0,24.53,,,N,NaN,5
1,ATOM,2,,H1,,LEU,,,1,,...,60.934,82.692,19.165,1.0,24.53,,,H,NaN,6
2,ATOM,3,,H2,,LEU,,,1,,...,61.257,81.809,17.821,1.0,24.53,,,H,NaN,7
3,ATOM,4,,H3,,LEU,,,1,,...,61.531,81.180,19.337,1.0,24.53,,,H,NaN,8
4,ATOM,5,,CA,,LEU,,,1,,...,62.939,82.566,18.769,1.0,24.53,,,C,NaN,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4749,ATOM,4750,,CD2,,PHE,,,318,,...,55.090,97.293,28.177,1.0,369.09,9,,,NaN,4754
4750,ATOM,4751,,HD2,,PHE,,,318,,...,54.841,96.773,27.267,1.0,369.09,9,,,NaN,4755
4751,ATOM,4752,,C,,PHE,,,318,,...,52.760,94.719,28.955,1.0,369.09,9,,,NaN,4756
4752,ATOM,4753,,OC1,,PHE,,,318,,...,52.032,95.705,29.207,1.0,369.09,9,,,NaN,4757
